In [1]:
import requests

def send_line_notify(notification_message):
    """
    LINEに通知する
    """
    with open('../line_notify_token', 'r') as f:
        line_notify_token = f.read()
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': f'{notification_message}'}
    requests.post(line_notify_api, headers = headers, data = data)

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf

N_CLASSES = 15
IMAGE_SIZE = 500
BATCH_SIZE = 32

train_data_augmer = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
    zoom_range=0.2,
    rotation_range=25,
    height_shift_range=0.2,
    width_shift_range=0.2,
    # channel_shift_range=0.1,
    brightness_range=[0.85,1.15],
)
valid_data_augmer = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0
)

train_generator = train_data_augmer.flow_from_directory(
    directory="../compe/train",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
valid_generator = valid_data_augmer.flow_from_directory(
    directory="../compe/val",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

def step_decay(epoch):
    lr = 0.001
    if(epoch >= 50):
        lr /= 10 /2
    if(epoch >= 100):
        lr /= 40/2
    return lr

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=60,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'data/temp/road_sign_sequential_{epoch:03d}_{val_loss:.4f}.h5',
        save_best_only=True
    ),
    # tf.keras.callbacks.TensorBoard(
    #     log_dir='logs',
    #     histogram_freq=1
    # ),
    tf.keras.callbacks.LearningRateScheduler(step_decay),
]



from tensorflow.keras.layers import Dense, Dropout, Conv2D, GlobalAveragePooling2D, MaxPooling2D, Flatten, Input, BatchNormalization, Resizing, Rescaling
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
for layer in vgg_model.layers:
    layer.trainable = False

x = vgg_model.output
x = Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x) 
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
# x = BatchNormalization()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
# x = BatchNormalization()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
# x = BatchNormalization()(x)
outputs = Dense(N_CLASSES, activation='softmax')(x)

EPOCHS = 200

model = tf.keras.Model(inputs=vgg_model.input, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
try:
    history = model.fit(train_generator, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=valid_generator, callbacks=callbacks)
except Exception as e:
    send_line_notify(f'stopped learning couse of {e}')
    print(e)
model.save('model.h5')



score = model.evaluate(valid_generator, verbose='0')
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# !tensorboard --logdir logs/fit
    

import pandas as pd

hist_df = pd.DataFrame(history.history)
hist_df[['loss', 'val_loss']].plot()
hist_df[['accuracy', 'val_accuracy']].plot()

print(model.summary())

2023-10-30 11:39:30.046808: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-30 11:39:30.073826: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 11:39:30.073855: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 11:39:30.073870: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 11:39:30.079410: I tensorflow/core/platform/cpu_feature_g

Found 225 images belonging to 15 classes.
Found 90 images belonging to 15 classes.


2023-10-30 11:39:31.609645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 11:39:31.613654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 11:39:31.613682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 11:39:31.615670: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 11:39:31.615700: I tensorflow/compile

Epoch 1/200


2023-10-30 11:39:36.870160: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2023-10-30 11:39:41.279251: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.29GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-10-30 11:39:54.079388: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f523c00bba0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-30 11:39:54.079422: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2023-10-30 11:39:54.083486: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-30 11:39:54.155001: I ./tensorflow/com

8/8 [==============================] - ETA: 0s - loss: 6.1144 - accuracy: 0.0578

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 43s 3s/step - loss: 6.1144 - accuracy: 0.0578 - val_loss: 11.2470 - val_accuracy: 0.0667 - lr: 0.0010
Epoch 2/200
8/8 [==============================] - 11s 1s/step - loss: 6.0615 - accuracy: 0.0444 - val_loss: 12.2220 - val_accuracy: 0.0667 - lr: 0.0010
Epoch 3/200
8/8 [==============================] - 11s 1s/step - loss: 5.9010 - accuracy: 0.0489 - val_loss: 10.9189 - val_accuracy: 0.0667 - lr: 0.0010
Epoch 4/200
8/8 [==============================] - ETA: 0s - loss: 5.7815 - accuracy: 0.0578

: 

In [ ]:
import numpy as np
import tensorflow as tf
import pathlib
import sklearn.metrics

def _main():
    idg = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0/255.0
	# 学習時のスケールと合わせる必要がある
    )
    gen_validation = idg.flow_from_directory(
        directory='../compe/val',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        shuffle=False
    )

    # 学習済みのモデルをロード
    model = tf.keras.models.load_model('model.h5')
    preds = model.predict(gen_validation, steps=len(gen_validation))
    # 検証データは以下の15クラスが6枚ずつ
    labels = [
        '01_stop',
        '02_dontEntry',
        '03_max_20',
        '04_max_30',
        '05_max_40',
        '06_max_50',
        '07_donotParking',
        '08_donotParkingStopping',
        '09_pedestrian',
        '10_bicycleAndPedestrianOnly',
        '11_pedestrianOnly',
        '12_oneWay',
        '13_goOnlyinDirectionOfArrow',
        '14_schoolRoad',
        '15_end'
    ]
    # 正解ラベルの作成
    y_label = []
    for label in labels:
        for i in range(6):
            y_label.append(label)

    # 推論ラベルの作成
    y_preds = []
    for p in preds:
        # 確信度が最大の値を推論結果とみなす
        label_idx = p.argmax()
        y_preds.append(labels[label_idx])
    
    # 混合行列を取得
    val_mat = sklearn.metrics.confusion_matrix(y_label, y_preds, labels=labels)
    rec_score = sklearn.metrics.recall_score(y_label, y_preds, average=None)
    print('再現率： ',rec_score)

    pre_score = sklearn.metrics.precision_score(y_label, y_preds, average=None)
    print('適合率： ', pre_score)
    print()

    f1_score = sklearn.metrics.f1_score(y_label, y_preds, average=None)
    print('F値   ： ', f1_score)

    rec_score_avg = sklearn.metrics.recall_score(y_label, y_preds, average="macro")
    print('再現率(平均)： ', rec_score_avg)
    pre_score_avg = sklearn.metrics.precision_score(y_label, y_preds, average="macro")
    print('適合率(平均)： ', pre_score_avg)
    f1_score_avg = sklearn.metrics.f1_score(y_label, y_preds, average="macro")
    print('F値(平均)   ： ', f1_score_avg)
    print()

    acc_score = sklearn.metrics.accuracy_score(y_label, y_preds)
    print('正解率： ', acc_score)
    send_line_notify(f'ended by {acc_score}')

if __name__ == '__main__':
    _main()


0.5024752616882324
0.5668317079544067
0.5891088843345642

In [ ]:
test_data_augmer = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0
)

test_generator = test_data_augmer.flow_from_directory(
    directory="../Test_images",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
score = model.evaluate(test_generator, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])